In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os 
from glob import glob
from tqdm import tqdm
import json
import cv2
import shutil


In [ ]:
def polygon2mask(polygon, mask, value):
    # polygon이 None이거나 비어있는 경우 처리
    if polygon[0][0] is None or len(polygon) == 0:
        return mask
    
    # polygon 내부에 None 값이 있는지 확인하고 필터링
    try:
        # None 값이 포함된 좌표 제거
        filtered_polygon = []
        for coord in polygon:
            if coord is not None and len(coord) == 2:
                if coord[0] is not None and coord[1] is not None:
                    filtered_polygon.append(coord)
        
        if len(filtered_polygon) < 3:  # 폴리곤을 만들기 위해 최소 3개 점 필요
            return mask
            
        polygon_array = np.array(filtered_polygon).reshape((-1, 1, 2)).astype(np.int32)
        mask_copy = np.ascontiguousarray(mask)
        cv2.fillPoly(mask_copy, [polygon_array], value)
        return mask_copy
        
    except (ValueError, TypeError) as e:
        print(f"Warning: Invalid polygon data - {e}")
        return mask

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
#Stomach
integrated_class={
    "NT_stroma":"stroma",
    "NT_Muscle":"stroma",
    "NT_immune":"immune",
    "NT_epithelial":"Normal",
    "NT_gastritis":"Normal",
    "NT_intestinal_metaplasia":"Normal",
    "Tumor":"tumor",
    "Tumor_diffuse":"tumor",
    "Tumor_intestinal":"tumor"
    
}
image_list=glob('../../data/NIPA/ST*/*.jpeg')
json_list=[f.replace('.jpeg','.json') for f in image_list]
class_count=4
image_path='../../data/NIPA/stomach/images/'
mask_path='../../data/NIPA/stomach/masks/'
create_directory(image_path)
create_directory(mask_path)
for i in tqdm(range(len(image_list))):
    image=Image.open(image_list[i])
    width, height=image.size
    json_file=json.load(open(json_list[i]))
    mask=np.zeros((height,width,class_count+1),dtype=np.uint8)
    for j in range(len(json_file['content']['file']['objects'])):
        label_name=json_file['content']['file']['objects'][j]['label_nm']
        if label_name=="Cell_nucleus":
            continue
        integrated_label=integrated_class[label_name]
        polygon=json_file['content']['file']['objects'][j]['coordinate']
        if integrated_label=='stroma':
            temp_mask = mask[:,:,1].copy()
            mask[:,:,1]=polygon2mask(polygon,temp_mask,1)
        elif integrated_label=='immune':
            temp_mask = mask[:,:,2].copy()
            mask[:,:,2]=polygon2mask(polygon,temp_mask,1)
        elif integrated_label=='Normal':
            temp_mask = mask[:,:,3].copy()
            mask[:,:,3]=polygon2mask(polygon,temp_mask,1)
        elif integrated_label=='tumor':
            temp_mask = mask[:,:,4].copy()
            mask[:,:,4]=polygon2mask(polygon,temp_mask,1)
    mask[:,:,0]=1-(mask[:,:,1]|mask[:,:,2]|mask[:,:,3]|mask[:,:,4])
    total_mask = mask.sum(axis=-1) 
    if total_mask.max()>1:
        overlap_indices = np.where(total_mask>1)
        for idx in zip(*overlap_indices):
            if mask[idx[0], idx[1], 1] == 1 and mask[idx[0], idx[1], 2] == 1:
                # Stroma and Immune overlap
                mask[idx[0], idx[1], 1] = 0 
            elif mask[idx[0], idx[1], 1] == 1 and mask[idx[0], idx[1], 3] == 1:
                # Stroma and Epithelial overlap
                mask[idx[0], idx[1], 1] = 0 
            elif mask[idx[0], idx[1], 1] == 1 and mask[idx[0], idx[1], 4] == 1:
                # Stroma and Tumor overlap
                mask[idx[0], idx[1], 1] = 0 
            elif mask[idx[0], idx[1], 2] == 1 and mask[idx[0], idx[1], 3] == 1:
                # Immune and Epithelial overlap
                mask[idx[0], idx[1], 3] = 0 
            elif mask[idx[0], idx[1], 2] == 1 and mask[idx[0], idx[1], 4] == 1:
                # Immune and Tumor overlap
                mask[idx[0], idx[1], 4] = 0 
            elif mask[idx[0], idx[1], 3] == 1 and mask[idx[0], idx[1], 4] == 1:
                # Epithelial and Tumor overlap  
                mask[idx[0], idx[1], 3] = 0
    shutil.copy(image_list[i],image_path+os.path.basename(image_list[i]))
    np.save(mask_path+os.path.basename(image_list[i]).replace('.jpeg','.npy'),mask)
    
        
                


In [ ]:
#breast
integrated_class={
    "TP_invasive": "Tumor",
    "TP_in_situ": "Tumor",
    "Tumor": "Tumor",
    "Stroma": "Stroma",
    "NT_stroma": "Normal",
    "NT_epithelial": "Normal",
    "NT_immune": "Immune"
}
image_list=glob('../../data/NIPA/BR*/*.jpeg')
json_list=[f.replace('.jpeg','.json') for f in image_list]
class_count=4
image_path='../../data/NIPA/breast/images/'
mask_path='../../data/NIPA/breast/masks/'
create_directory(image_path)
create_directory(mask_path)
for i in tqdm(range(len(image_list))):
    image=Image.open(image_list[i])
    width, height=image.size
    json_file=json.load(open(json_list[i]))
    mask=np.zeros((height,width,class_count+1),dtype=np.uint8)
    for j in range(len(json_file['content']['file']['objects'])):
        label_name=json_file['content']['file']['objects'][j]['label_nm']
        if label_name=="Cell_nucleus":
            continue
        integrated_label=integrated_class[label_name]
        polygon=json_file['content']['file']['objects'][j]['coordinate']
        if integrated_label=='Stroma':
            temp_mask = mask[:,:,1].copy()
            mask[:,:,1]=polygon2mask(polygon,temp_mask,1)
        elif integrated_label=='Immune':
            temp_mask = mask[:,:,2].copy()
            mask[:,:,2]=polygon2mask(polygon,temp_mask,1)
        elif integrated_label=='Normal':
            temp_mask = mask[:,:,3].copy()
            mask[:,:,3]=polygon2mask(polygon,temp_mask,1)
        elif integrated_label=='Tumor':
            temp_mask = mask[:,:,4].copy()
            mask[:,:,4]=polygon2mask(polygon,temp_mask,1)
    mask[:,:,0]=1-(mask[:,:,1]|mask[:,:,2]|mask[:,:,3]|mask[:,:,4])
    total_mask = mask.sum(axis=-1) 
    if total_mask.max()>1:
        overlap_indices = np.where(total_mask>1)
        for idx in zip(*overlap_indices):
            if mask[idx[0], idx[1], 1] == 1 and mask[idx[0], idx[1], 2] == 1:
                # Stroma and Immune overlap
                mask[idx[0], idx[1], 1] = 0 
            elif mask[idx[0], idx[1], 1] == 1 and mask[idx[0], idx[1], 3] == 1:
                # Stroma and Epithelial overlap
                mask[idx[0], idx[1], 1] = 0 
            elif mask[idx[0], idx[1], 1] == 1 and mask[idx[0], idx[1], 4] == 1:
                # Stroma and Tumor overlap
                mask[idx[0], idx[1], 1] = 0 
            elif mask[idx[0], idx[1], 2] == 1 and mask[idx[0], idx[1], 3] == 1:
                # Immune and Epithelial overlap
                mask[idx[0], idx[1], 3] = 0 
            elif mask[idx[0], idx[1], 2] == 1 and mask[idx[0], idx[1], 4] == 1:
                # Immune and Tumor overlap
                mask[idx[0], idx[1], 4] = 0 
            elif mask[idx[0], idx[1], 3] == 1 and mask[idx[0], idx[1], 4] == 1:
                # Epithelial and Tumor overlap  
                mask[idx[0], idx[1], 4] = 0
    shutil.copy(image_list[i],image_path+os.path.basename(image_list[i]))
    np.save(mask_path+os.path.basename(image_list[i]).replace('.jpeg','.npy'),mask)